# Nettoyez et analysez votre jeu de données Nettoyez et analysez votre jeu de données 

## Nettoyer vos données avec python

### Detecter les erreurs

In [95]:
# import des librairies dont nous aurons besoin
import pandas as pd
import numpy as np
import re


In [96]:
# chargement et affichage des données
data = pd.read_csv("personnes.csv")
display(data)


,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,24,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


#### Valeurs manquantes

In [97]:
# Detecter les valeurs manquante (les NULL)
print(data.isnull().sum())


prenom            0
email             0
date_naissance    1
pays              1
taille            0
dtype: int64


#### Doublons

In [98]:
# Detecter les doublons
data.loc[data["email"].duplicated(keep=False), :]


,prenom,email,date_naissance,pays,taille
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


### Traitez les erreurs : 

#### Theorie

`# Nouvelle dataframe :`

`data['nom_colonne'] = nouvelle_colonne`

`mask = # condition à vérifier pour cibler spécifiquement certaines lignes`

`data.loc[mask, 'ma_colonne'] = nouvelles_valeurs`

#### Traitez les pays :

In [99]:
display(data["pays"])


0           France
1              NaN
2    Côte d'ivoire
3           France
4       Madagascar
5               24
6            Bénin
Name: pays, dtype: object

- Le probleme etait que il y a la valeur 24 qui ne correspond pas a un pays
- Pour regler cette erreurs nous allons creer un dictionnaire ou nous allons mettre nos pays
- Nous alons donc verifier si le pays correspond ou non a un pays du dictionnaire
- Si il correspond alors tout est bon
- Si ça correspond pas alors on met un ***NaN***

In [100]:
VALID_COUNTRIES = ["France", "Côte d'ivoire", "Madagascar", "Bénin", "Allemagne", "USA"]
mask = ~data["pays"].isin(VALID_COUNTRIES)
data.loc[mask, "pays"] = np.NaN
display(data["pays"])


0           France
1              NaN
2    Côte d'ivoire
3           France
4       Madagascar
5              NaN
6            Bénin
Name: pays, dtype: object

On remarque que la valeur ***24*** qui n'etait pas dans le dictionnaire er devnu la valeur ***Nan***

#### Traitez les e-mails

In [101]:
data["email"]


0                         leila@example.com
1                    samuel_329@example.com
2                  choupipoune@supermail.eu
3    marco23@example.com, mc23@supermail.eu
4                   helloworld@supermail.eu
5                    hanna2019@supermail.eu
6                    samuel_329@example.com
Name: email, dtype: object

Au tour des e-mails, maintenant ! Le problème avec cette colonne, c'est qu'il y a parfois 2 adresses e-mail par ligne. Nous ne souhaitons prendre que la première

In [102]:
data["email"] = data["email"].str.split(",", n=1, expand=True)[0]
display(data["email"])


0           leila@example.com
1      samuel_329@example.com
2    choupipoune@supermail.eu
3         marco23@example.com
4     helloworld@supermail.eu
5      hanna2019@supermail.eu
6      samuel_329@example.com
Name: email, dtype: object

Nous avons plus que un email par ligne

#### Traitez les tailles

In [103]:
display(data["taille"])


0    1.49m
1    1.67m
2    153cm
3    1.65m
4    1.34m
5    3.45m
6    1.45m
Name: taille, dtype: object

On remarque une taille impossible et un taille en cm

Ici nous decidons de garder seulement les taille en m. Et puis de donner les taille manquantes grace a la moyenne des autre tailles. Nous ne faisons pax la conversion cm -> m

In [104]:
data["taille"] = data["taille"].str[:-1]
data["taille"] = pd.to_numeric(data["taille"], errors="coerce")
display(data["taille"])


0    1.49
1    1.67
2     NaN
3    1.65
4    1.34
5    3.45
6    1.45
Name: taille, dtype: float64

In [105]:
data.loc[data["taille"].isnull(), "taille"] = data["taille"].mean()
data["taille"] = data["taille"].round(2)
display(data["taille"])


0    1.49
1    1.67
2    1.84
3    1.65
4    1.34
5    3.45
6    1.45
Name: taille, dtype: float64

On remarque que il rest la taille de 3.45 m 
On va essayrer de la traiter

In [106]:
data.loc[(data["taille"] <= 1.0) | (data["taille"] >= 2.5), "taille"] = data[
    "taille"
].mean()
data["taille"] = data["taille"].round(2)
display(data["taille"])


0    1.49
1    1.67
2    1.84
3    1.65
4    1.34
5    1.84
6    1.45
Name: taille, dtype: float64

On remarque que nous avons remplacer la valeur trop haute

#### Triez les dates

Nous allons spécifier un certain format de date, et transformer notre variable dans un type adéquat. Les formats non adequat vont etre transformer en NaN grace à ***coerce*** comme pour les tailles

In [107]:
data["date_naissance"] = pd.to_datetime(
    data["date_naissance"], format="%d/%m/%Y", errors="coerce"
)
display(data["date_naissance"])


0   1990-01-23
1   2001-09-20
2          NaT
3   1978-02-10
4   2008-03-05
5   1970-01-01
6          NaT
Name: date_naissance, dtype: datetime64[ns]

### Exercice chapitre 2

In [108]:
# importation des données
df_operation = pd.read_csv("operations.csv")
df_operation.head()


,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE


<p>Instruction</p>
<blockquote> Plusieurs erreurs se sont glissées dans ce jeu de données. Votre mission, si toutefois vous l'acceptez, va être de les trouver et de proposer des solutions adéquates pour les gérer. Vous aurez besoin dans tous les cas de nettoyer votre fichier avant de passer à la suite !</blockquote>

**Premiere étape : on enleve les doublons**

In [109]:
df_operation = df_operation.loc[df_operation.duplicated(keep="first") == False, :]


**2 : On supprime les nan** 

In [110]:
# display(df_operation.loc[df_operation.isnull(),:])
print(df_operation.isnull().sum())


date_operation    0
libelle           0
montant           2
solde_avt_ope     0
categ             1
dtype: int64


- valeur manquante en catégorie : on va mettre autre

- valeur manquante en montant on peut la calculer

In [111]:
# Categ
display(df_operation.loc[df_operation["categ"].isnull(), :])
df_operation.loc[df_operation["categ"].isnull(), "categ"] = "AUTRE"
display(df_operation.loc[df_operation["categ"].isnull(), :])


,date_operation,libelle,montant,solde_avt_ope,categ
156,2023-07-06,PRELEVEMENT XX TELEPHONE XX XX,-36.48,3295.68,NaN


,date_operation,libelle,montant,solde_avt_ope,categ


On recalculera les montants plus tard lorsqu'on verifiera les calcules

**3: traitez les dates opérations**
- verifier le format
- Si pas le bon format on le rend `null`

In [112]:
pd.to_datetime(df_operation["date_operation"], errors="coerce")


0     2023-03-31
1     2023-04-03
2     2023-04-03
3     2023-04-03
4     2023-04-03
         ...    
304   2023-10-05
305   2023-10-05
306   2023-10-06
307   2023-10-06
308   2023-10-06
Name: date_operation, Length: 308, dtype: datetime64[ns]

**4 : Gerer les libellées**

Je ne voit pas d'erreur dans les libellé et je ne vois pas comment les gerer.


**5 : Verifier que le montant avant opertion et le solde concorde**

Il faut verifier $ \forall i \in [\![1:n]\!], \ montant[i]=solde\_avt\_ope[i+1]-solde\_avt\_ope[i]$

In [113]:
array_operation = df_operation.to_numpy()
# montant : [i,2] / solde_avt_ope : [i,3]
for i in range(0, array_operation.shape[0] - 1):
    print(round(array_operation[i + 1, 3] - array_operation[i, 3], 2))
    print(array_operation[i, 2])
    if pd.isna(array_operation[i, 2]):
        array_operation[i, 2] = round(
            array_operation[i + 1, 3] - array_operation[i, 3], 2
        )
    if array_operation[i, 2] != round(
        array_operation[i + 1, 3] - array_operation[i, 3], 2
    ):
        array_operation[i + 1, 3] = round(
            array_operation[i, 2] + array_operation[i, 3], 2
        )



---
0
-1.44
-1.44

---
1
-24.0
-24.0

---
2
-73.0
-73.0

---
3
676.0
676.0

---
4
4.8
4.8

---
5
-14.39
-14.39

---
6
-15.2
-15.2

---
7
-12.0
-12.0

---
8
-7.02
-7.02

---
9
-6.8
-6.8

---
10
-1.84
-1.84

---
11
992.8
992.8

---
12
-24.0
-24.0

---
13
1.6
1.6

---
14
-84.4
-84.4

---
15
-12.42
-12.42

---
16
-6.0
-6.0

---
17
-20.64
-20.64

---
18
-1.6
-1.6

---
19
-42.42
-42.42

---
20
-2.7
-2.7

---
21
836.0
836.0

---
22
-7.56
-7.56

---
23
869.44
869.44

---
24
-48.0
-48.0

---
25
-28.96
-28.96

---
26
-280.0
-280.0

---
27
-1.6
-1.6

---
28
-8.8
-8.8

---
29
-0.8
-0.8

---
30
-345.0
-345.0

---
31
1.6
1.6

---
32
-10.32
-10.32

---
33
-8.72
-8.72

---
34
-24.0
-24.0

---
35
-74.58
-74.58

---
36
-1.7
-1.7

---
37
-13.6
-13.6

---
38
-7.2
-7.2

---
39
-10.43
-10.43

---
40
-7.84
-7.84

---
41
-11.64
-11.64

---
42
-9.2
-9.2

---
43
-32.67
-32.67

---
44
-10.0
-10.0

---
45
-1.9
-1.9

---
46
-9.11
-9.11

---
47
-1.4
-1.4

---
48
-36.0
-36.0

---
49
-12.24
-12.24

---
50
47.2
47.2
